# Download the txt file

In [1]:
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x7f827e9934c0>)

# Read the txt file

In [2]:
with open(file_path,'r',encoding='utf-8') as f:
    raw_text = f.read()
print("Total Number of characters:", len(raw_text))
print(raw_text[:100])

Total Number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


In [3]:
import re

# Split up based on whitespaces

In [4]:
text = "Hello World! This is a test."
result = re.split(r'(\s)',text)
print(result)

['Hello', ' ', 'World!', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test.']


#Split up the text on period(.), comma(,),  exclamation(!), and Whitspace.

In [5]:
result = re.split(r'([!,.]|\s)',text)
print(result)

['Hello', ' ', 'World', '!', '', ' ', 'This', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


<u>**.strip()** </u> -> removes any extra whitespace (trailing or leading) from any string.


In [6]:
word = " Hello World! "
print("Length of word before striping off the Whitspace:",len(word))
print(word)
print("Length of word after striping off the Whitspace:", len(word.strip()))
print (word.strip())

Length of word before striping off the Whitspace: 14
 Hello World! 
Length of word after striping off the Whitspace: 12
Hello World!


# Remove Redundant Whitespace

In [7]:
result = re.split(r'([!.,]|\s)',text)
res = [token for token in result if token.strip()]
print(res)

['Hello', 'World', '!', 'This', 'is', 'a', 'test', '.']


# Try it on the "The Verdict.txt" file

In [8]:
preprocessed_text = re.split(r'([!.,:;_?"\'()]|--|\s)',raw_text)
preprocessed_text = [token for token in preprocessed_text if token.strip()]
print(preprocessed_text[:30])
print(len(preprocessed_text))

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
4690


# Extract Unique tokens using set()

In [9]:
unique_tokens = set(preprocessed_text)
print("Number of Unique Tokens:", len(unique_tokens))

Number of Unique Tokens: 1130


# sort the list of tokens lexicographically

In [10]:
all_tokens = sorted(unique_tokens)
print(all_tokens[:22])

['!', '"', "'", '(', ')', ',', '--', '.', ':', ';', '?', 'A', 'Ah', 'Among', 'And', 'Are', 'Arrt', 'As', 'At', 'Be', 'Begin', 'Burlington']


In [11]:
vocab = { tok: idx for idx,tok in enumerate(all_tokens)}
for tok,idx in vocab.items():
    print(tok,idx)
    if idx == 21:
        break

! 0
" 1
' 2
( 3
) 4
, 5
-- 6
. 7
: 8
; 9
? 10
A 11
Ah 12
Among 13
And 14
Are 15
Arrt 16
As 17
At 18
Be 19
Begin 20
Burlington 21


# Creating Tokenizer class

In [12]:
class SimpleTokenizerV1:
  def __init__(self,vocab):
    self.word_to_index = vocab
    self.index_to_word = {i:word for word,i in vocab.items()}
  def encode(self,text):  # returns integer ids from the mapping (word to id) for a given given text within the whole story.
    text = re.split(r'([,.?!_()";:\']|--|\s)',text)
    text = [word.strip() for word in text if word.strip()]
    ids = [self.word_to_index[word] for word in text]
    return ids
  def decode(self, ids):  # return the converted string from the ids
    str1 = ' '.join([self.index_to_word[id] for id in ids])
    str1 = re.sub(r'\s+([,.?!"()\'])', r'\1',str1) # Replace spaces before the specified punctuations with the punctuation mark(, or . or ; or : or " or ? or ! or ')
    return str1


In [13]:
tokenizer = SimpleTokenizerV1(vocab)
text = "It's the last he painted, you know, Mrs. Gisburn said with pardonable pride."
ids = tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))


[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 67, 7, 38, 851, 1108, 754, 793, 7]
It' s the last he painted, you know, Mrs. Gisburn said with pardonable pride.


# Adding special context tokens for detecting Unknown Words for the vocabulary and indicating the separation between different documents

In [16]:
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [17]:
len(vocab.items())


1132

In [18]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [19]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [23]:
ids = tokenizer.encode(text)
print(ids)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [24]:
tokenizer.decode(ids)

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'